# 1. The Smart Cart Problem solving using Q-Learning

## 1.0 install the gym environment 
- conda install -c conda-forge gym

### 1.1 load the game environment and render what it looks like

In [1]:
import gym
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## 1.2 reset environment to a new, random state and render what it looks like

In [2]:
env.reset()
env.render()

print("The Action Space {}".format(env.action_space))
print("The State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

The Action Space Discrete(6)
The State Space Discrete(500)


## 1.3 Representations
- The filled square represents the taxi. Yellow means there's no passenger, green means there's a passenger on the taxi
- The pipe ("|") represents a wall that the taxi cannot cross.
- R, G, Y, B are the possible pickup and destination locations. The blue letter means the current passenger pick-up location, and the purple letter represents the current destination.
- The action number is from 0-5 where 0 = south, 1 = north, 2 = east, 3 = west, 4 = pickup, 5 = dropoff

As verified by the prints, we have an Action Space of size 6 and a State Space of size 500 = 5 * 5 parking lots, 4 destinations and (4+1) passenger locations.

## 1.4 Encode the state and give it to the environment to render in Gym

In [3]:
# The parameters of encode is (taxi row, taxi column, passenger index, destination index)
state = env.encode(2, 1, 2, 0)
print("State:", state)

env.s = state
env.render()

State: 228
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## 1.5 The reward Table

When the Taxi environment is created, there is an initial Reward table that's also created, called `P`. We can think of it like a matrix that has the number of states as rows and number of actions as columns, i.e. a states × actions matrix.

- The 0-5 corresponds to the actions (south, north, east, west, pickup, dropoff) the taxi can perform at our current state in the illustration.
- In this env, probability is always 1.0.
- The nextstate is the state we would be in if we take the action at this index of the dict.
- All the movement actions have a -1 reward and the pickup/dropoff actions have -10 reward in this particular state. If we are in a state where the taxi has a passenger and is on top of the right destination, we would see a reward of 20 at the dropoff action (5).
- done is used to tell us when we have successfully dropped off a passenger in the right location. Each successfull dropoff is the end of an episode.

In [4]:
# the structure of the dictionary P is {action: [(probability, nextstate, reward, done)]}
env.P[228]

{0: [(1.0, 328, -1, False)],
 1: [(1.0, 128, -1, False)],
 2: [(1.0, 248, -1, False)],
 3: [(1.0, 208, -1, False)],
 4: [(1.0, 228, -10, False)],
 5: [(1.0, 228, -10, False)]}

## 1.6 Solving the environment without Reinforcement Learning

### 1.6.1 Process

- Create an infinite loop which runs until one passenger reaches one destination (one episode), or when the received reward is 20. 

- The env.action_space.sample() method automatically selects one random action from set of all possible actions.

In [7]:
# set environment to illustration's state
env.s = 328

epochs = 0
penalties, reward = 0, 0
frames = []
done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )
    epochs += 1
    
print("The Timesteps taken: {}".format(epochs))
print("The Penalties incurred: {}".format(penalties))

The Timesteps taken: 4999
The Penalties incurred: 1632


### 1.6.2 Show all the states for each step

In [8]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(f"{frame['frame']}") 
        print(f"The Timestep: {i + 1}")
        print(f"The State: {frame['state']}")
        print(f"The Action: {frame['action']}")
        print(f"The Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

The Timestep: 4999
The State: 0
The Action: 5
The Reward: 20


### 1.6.3 Result Analysis

Because we never learn from past experience, it takes too many steps. Although we run this over and over, it will never be optimized. The agent has no memory of which action was best for each state, which is exactly what Reinforcement Learning will do for us. Next, we want to use Reinforcement Learning to get the optimization solution.

## 1.7 Reinforcement Learning Solution

Summing up the Q-Learning Process:
- Initialize the Q-table by all zeros.
- Start exploring actions: For each state, select any one among all possible actions for the current state (S).
- Travel to the next state (S') as a result of that action (a).
- For all possible actions from the state (S') select the one with the highest Q-value.
- Update Q-table values using the equation.
- Set the next state as the current state.
- If goal state is reached, then end and repeat the process.

### 1.7.1 Initialize the Q-table to a 500×6 matrix of zeros

The Q table (0,1,2,3,4,5)
- 0 represents South
- 1 represents North
- 2 represents East
- 3 represents West
- 4 represents pickup
- 5 represents dropoff

In [11]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])
print(q_table)
print(q_table.shape)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
(500, 6)


### 1.7.2 Training the agent
- Create the training algorithm and update this Q-table as the agent explores the environment over thousands of episodes.
- Decide whether to pick a random action or to exploit the already computed Q-values by using the epsilon value and comparing it to the random.uniform(0, 1) function, which returns an arbitrary number between 0 and 1.
- Execute the chosen action in the environment to obtain the next_state and the reward from performing the action.
- Calculate the maximum Q-value for the actions corresponding to the next_state, and update our Q-value to the new_q_value.

In [12]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

all_epochs = []
all_penalties = []

for i in range(1, 150001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])

        next_state, reward, done, info = env.step(action) 
        
        Pre_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * Pre_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value
        
        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 150000
Training finished.



In [13]:
q_table[228]

array([ -2.3639511 ,  -2.3639511 ,  -2.3639511 ,  -2.1220864 ,
       -11.27325178, -11.27325176])

### 1.7.3 Result Analysis

- The Q-table has been established over 150,000 episodes, and the Q-values are shwon as above.
- The maximum Q value is "west" (-2.1220864), so it looks like Q-Learning has effectively learned the best action to take in the illustration state!

### 1.7.4 evaluating the agent

- Needn't to explore actions any further and the next action is always selected using the best Q-value.

- From the evaluation, the agent's performance improved significantly and it incurred no penalties, meaning it performed the correct pickup/dropoff actions with 500 different passengers.

In [14]:
total_epochs, total_penalties = 0, 0
episodes = 500

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 500 episodes:
Average timesteps per episode: 13.004
Average penalties per episode: 0.0
